#### LLM Text extractor

In [2]:
from dotenv import load_dotenv
import os
from langchain.chains import create_extraction_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
import PyPDF2

##### Load my api key

In [3]:
# Load the .env file
load_dotenv()

# Accessing variables
myapikey = os.getenv('OPENAI_API_KEY')

In [4]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=myapikey, temperature=0, model_name="gpt-4")

c:\Users\josef\Documents\projects\pilotchatbot\pilotbotenv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


#### PDFs document loading

In [5]:
# Define the folder path to the PDF files
folder_path = '../data/cz_vfr_manual/'

# Get a list of PDF files in the directory
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
pdf_paths = [os.path.join(folder_path, file) for file in pdf_files]

#### Text splitter settings

In [6]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
from langchain_openai import OpenAIEmbeddings

tokenizer_name = tiktoken.encoding_for_model('gpt-4')
tokenizer_name.name

# create the length function
def tiktoken_len(text):
    tokens = tokenizer_name.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=128,
    length_function=tiktoken_len,
    separators=["."],
    is_separator_regex=False,
)

def extract_cz_tags_from_filename(filename):
    # Extract the first occurrence of ICAO code from the filename
    match = re.search(r'lk[a-z]{2,6}', filename)
    
    # Return the found code in uppercase if there's a match, otherwise return None
    return match.group().upper() if match else None

OpenAI Embeddings model

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=myapikey
)

c:\Users\josef\Documents\projects\pilotchatbot\pilotbotenv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
from langchain.document_loaders import PyPDFLoader

chunked_texts_with_tags = []
# Iterate over each PDF file
for path in pdf_paths:
    # Extract the filename from the path
    filename = os.path.basename(path)

    # Extract tags from the filename
    pdf_tags = extract_cz_tags_from_filename(filename)
    #print(pdf_tags)
    # Open and read the PDF file
    pdf_reader = PyPDF2.PdfReader(path)
    full_text = f"Document: {filename}\n"  # Start with the filename

    for page in pdf_reader.pages:
        # if less than 200 char then skip
        if len(page.extract_text()) < 200:
            continue
        else:
            page_text = page.extract_text()
            if page_text:  # Check if text extraction is successful
                page_text = page_text.replace('\n', '')
                full_text += page_text
            else:
                full_text += f"[No text extracted from this page]\n"
            
            chunks = text_splitter.split_text(full_text)
            # Add tags to each chunk
            for chunk in chunks:
                chunk_with_tags = f"ICAO airport code: {(pdf_tags)}, {chunk}"
                chunked_texts_with_tags.append(chunk_with_tags)


In [12]:
#res = embed.embed_documents(chunked_texts_with_tags)

In [9]:
from langchain.vectorstores import Chroma
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory="../data/vectordb/")

In [10]:
db2 = vectorstore.from_texts(chunked_texts_with_tags, embed, persist_directory="../data/vectordb/")

Pydantic

https://github.com/jiggy-ai/pydantic-chatcompletion

In [81]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer_name = tiktoken.encoding_for_model('gpt-4')
tokenizer_name.name

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=128,
    length_function=tiktoken_len,
    separators=["."]
)

pdf_reader = PyPDF2.PdfReader('../data/cz_vfr_manual/ad-lklt_text_cz.pdf')
full_text = f"Document: {'ad-lklt_map_cz'}\n"  # Start with the filename

for page in pdf_reader.pages:
    # if less than 200 char then skip
    if len(page.extract_text()) < 200:
        continue
    page_text = page.extract_text()
    if page_text:  # Check if text extraction is successful
        page_text = page_text.replace('\n', '')
        full_text += page_text
    else:
        full_text += f"[No text extracted from this page]\n"
    
    texts = text_splitter.split_text(full_text)

In [82]:
    texts

['Document: ad-lklt_map_cz\n05 OCT 23 (1) Letňany VFR-AD-LKLT-TEXT- 1VFR příručka - Česká republika1 PRAVIDLA A OMEZENÍ MÍSTNÍHO LETOVÉHO PROVOZU1.1 Všeobecně1.1.1 Všichni uživatelé LKLT jsou před jeho využitím povinni se seznámit s letištním řádemLKLT, který je v aktuální podobě k dispozici na webových stránkách www.letnany-airport.cz.1.1.2 Letiště LKLT leží v MCTR Kbely a let po okruhu zasahuje do CTR Ruzyně.1.1.3 Jsou zřízeny prostory TRA GA TRAPRLT1 (Letňany 1N), TRAKBLT1 (Letňany 1S) aTRAKBLT1E (Letňany 1E) viz. AIP ČR, ENR 5.5.5. Informace o aktivaci a rozsahuvyužívání TRA GA lze získat telefonicky nebo na provozním kmitočtu APP Praha, TWRRuzyně, MAPP/MTWR Kbely, stanoviště Poskytování informací známému provozuLetňany nebo FIC Praha.1.1.4 Pokud je předmětný prostor TRA GA aktivován, aplikují se v něm podmínky a pravidlavzdušného prostoru třídy G. Prostory TRA GA jsou zároveň oblastí s povinným radiovýmspojením (RMZ).1.1',
 '.1.1.4 Pokud je předmětný prostor TRA GA aktivován, apli

In [55]:
tokenizer = tiktoken.get_encoding(tokenizer_name.name)

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [58]:
 texts = text_splitter.split_text(full_text)

In [59]:
texts

['Document: ad-lklt_map_cz\n05 OCT 23 (1) Letňany VFR-AD-LKLT-TEXT- 1\nVFR příručka - Česká republika1 PRAVIDLA A OMEZENÍ MÍSTNÍHO LETOVÉHO PROVOZU\n1.1 Všeobecně\n1.1.1 Všichni uživatelé LKLT jsou před jeho využitím povinni se seznámit s letištním řádem\nLKLT, který je v aktuální podobě k dispozici na webových stránkách www.letnany-\nairport.cz.\n1.1.2 Letiště LKLT leží v MCTR Kbely a let po okruhu zasahuje do CTR Ruzyně.\n1.1.3 Jsou zřízeny prostory TRA GA TRAPRLT1 (Letňany 1N), TRAKBLT1 (Letňany 1S) a\nTRAKBLT1E (Letňany 1E) viz. AIP ČR, ENR 5.5.5. Informace o aktivaci a rozsahu\nvyužívání TRA GA lze získat telefonicky nebo na provozním kmitočtu APP Praha, TWR\nRuzyně, MAPP/MTWR Kbely, stanoviště Poskytování informací známému provozu\nLetňany nebo FIC Praha.\n1.1.4 Pokud je předmětný prostor TRA GA aktivován, aplikují se v něm podmínky a pravidla\nvzdušného prostoru třídy G. Prostory TRA GA jsou zároveň oblastí s povinným radiovým\nspojením (RMZ).\n1.1.5 Mimo provozní dobu stanovišt

In [8]:
# Schema
ad_schema = {
    "properties": {
        "icao_airport": {"type": "string"},
        "airport_name": {"type": "integer"},
        "airport_city": {"type": "string"},
        "opening_hours": {"type": "string"},
    },
    "required": ["icao_airport", "airport_name","airport_city"],
}
# Run chain
ad_llm = ChatOpenAI(temperature=0, model="gpt-4")

# Function to determine the document type
def get_doc_type(text):
    if text.lower().startswith("gen"):
        return "GEN"
    elif text.lower().startswith("enr"):
        return "ENR"
    elif text.lower().startswith("ad"):
        return "AD"
    return "Unknown"

# Function to extract data based on the document type
def extract_data(doc_type, full_text, filename):
    # unstructed text extraction
    if doc_type in ["GEN", "ENR"]:
        text_splitter = CharacterTextSplitter(
          separator="\n",
          chunk_size=512,
          chunk_overlap=128
        )
        document = text_splitter.create_documents([full_text], metadatas=[{"filename":filename}])

        return document
    elif doc_type == "AD":
        # chain
        print(full_text)
        _input = prompt.format_prompt(query=full_text)
        model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)
        output = model(_input.to_string())
        parser.parse(output)
        #chain = create_extraction_chain(ad_schema, promt, ad_llm)
        #chain_output = chain.run(text)
        #print(chain_output)
        # text splitter
        text_splitter = CharacterTextSplitter(
          separator="\n",
          chunk_size=512,
          chunk_overlap=128
        )
        document = text_splitter.create_documents([full_text], metadatas=[{"filename":filename}])

        pass  # Implement the extraction logic here

# List to store all documents
documents = []

# Iterate over each PDF file
for path in pdf_paths:
    # Extract the filename from the path
    filename = os.path.basename(path)

    # Open and read the PDF file
    pdf_reader = PyPDF2.PdfReader(path)
    full_text = ""

    for page in pdf_reader.pages:
        page_text = page.extract_text()
        full_text += page_text + "\n" if page_text else "[No text extracted from this page]\n"

    # Determine the document type
    doc_type = get_doc_type(filename)
    #doc =  Document(page_content=full_text, metadata={"source": filename})
    # Extract structured data based on document type
    structured_data = extract_data(doc_type, full_text, filename)
    # Create a document dictionary
    document = {
        'filename': filename,
        'type': doc_type,
        'structured_data': structured_data
    }

    # Add the document to the list
    documents.append(document)

02 NOV 23 (1) Liberec VFR-AD-LKLB-VOC
VFR příručka - Česká republika
LKLB Liberec
§Neveřejné mezinárodní letiště VFR den, výsadková činnost
Liberec RADIO
122,605ARP:  50° 46' 06" N, 15° 01' 30" E
2,5 km W Liberec
ELEV:  1329 ft / 405 m
Okruh:  2330 ft / 710 m AMSL
3° E/2010
! Hluková omezení
-Lety nad zástavbou města Liberce (s výjimkou letištních okruhů) provádět minimálně ve výšce 3300
ft/1000 m AMSL s trajektorií letu volenou tak, aby nedocházelo k opakovanému kroužení v jednom
prostoru.
V době Poskytování informací známému provozu je s ohledem na danou meteorologickou situaci a další
provozní podmínky nutno upřednostňovat přímá přiblížení a přímé odlety letadel bez zbytečných letů po
okruhu.
23 FEB 23 (1) Liberec VFR-AD-LKLB-ADC
VFR příručka - Česká republika
LKLB Liberec
Liberec RADIO
122,605RWYMagnetický
směrRozměry
RWYÚnosnost TORA TODA ASDA LDA
16 162° 1020 x 50 5700 kg / 0.5 MPa 1020 1080 1020 900
34 342° 900 x 50 5700 kg / 0.5 MPa 960 960 1020 1020
Statutární město Liberec
Ná

NameError: name 'prompt' is not defined

In [ ]:
documents

[{'filename': 'ad-lktb_text_cz.pdf', 'type': 'AD', 'structured_data': None},
 {'filename': 'ad-lktb_map_cz.pdf', 'type': 'AD', 'structured_data': None}]

In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage


In [ ]:
from google.colab import drive
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
import os
import re

"""
def query_chatgpt(icao_code):
    # Construct the prompt for querying information about the ICAO code

    messages = [
      SystemMessage(
          content="You are very brief, informative. I need assistent to do me a mapping - ICAO, airport name, region, city and some basic infromation about. All airports are in Czech republic, be careful about ICAO code, some airport are very local and please prefer as source soume VFR flight manual. Don´t make mistakes like LKBR is Brno airport, when is actually Broumov airport. Be very careful."
      ),
      HumanMessage(
          content=f"What airport has the ICAO code {icao_code}? Please provide the name, location, town and country."
      ),
    ]
    response = llm(messages)
    if response:
        # Assuming the last message in the response is the AI's reply
        response_text = response.content
        return response_text
    else:
        return ""
    # Extract the relevant information from the response
    # Note: You might need to parse the response to extract structured data
    return response

def extract_tags_from_filename(filename):
    # Extract ICAO codes from the filename
    icao_codes = re.findall(r'lk[a-z]{2,6}', filename)

    tags = []
    for code in icao_codes:
        airport_info = query_chatgpt(code)
        if airport_info:
            tags.append(airport_info)

    return tags
"""
# Iterate over each PDF file
for path in pdf_paths:
    # Extract the filename from the path
    filename = os.path.basename(path)

    # Extract tags from the filename
    #pdf_tags = extract_tags_from_filename(filename)
    #print(pdf_tags)
    # Open and read the PDF file
    pdf_reader = PyPDF2.PdfReader(path)
    full_text = f"Document: {filename}\n"  # Start with the filename

    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:  # Check if text extraction is successful
            full_text += page_text + "\n"
        else:
            full_text += "[No text extracted from this page]\n"
    """
    # Split the text into chunks using CharacterTextSplitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=300,
        length_function=len,
        is_separator_regex=False
    )

    chunks = text_splitter.create_documents([full_text])

    # Add tags to each chunk
    for chunk in chunks:
        chunk_with_tags = f"Tags: {', '.join(pdf_tags)}\n{chunk}"
        chunked_texts_with_tags.append(chunk_with_tags)
    """
#texts = text_splitter.create_documents([chunked_texts_with_tags])

KeyboardInterrupt: 

In [ ]:
chunked_texts_with_tags

["Tags: \npage_content='Document: gen_2_cz.pdf\\n01 MAY 14 (1) Deset tipů VFR-GEN-2- 1\\nVFR příručka - Česká republikaGEN-2 DESET TIPŮ JAK SE VYHNOUT PROBLÉMŮM\\n1)Uvědomte si, že dokonalá znalost leteckých předpisů, pravidel a postupů, spolu se správným\\nplánováním a předletovou přípravou, je základním předpokladem k bezpečnému provedení\\nletu!\\nAIP ČR: http://lis.rlp.cz/ais_data/www_main_control/frm_cz_aip.htm\\nLetecké předpisy: http://lis.rlp.cz/predpisy/predpisy/index.htm\\n2)Jestliže plánujete trať, pamatujte, že meteorologické nebo provozní podmínky vám nemusí\\numožnit dodržet původně plánovanou trať a proto nezapomeňte při výpočtech zohlednit\\ncelkově delší čas letu, zajistit dostatečnou zásobu paliva na delší trať a připravit si postupy\\npro přistání na náhradních letištích!\\n3)Tam, kde je to možné, neplánujte let v blízkosti horizontálních nebo vertikálních hranic\\nřízeného prostoru. Pokud tak potřebujete letět, buďte velmi opatrní. Malá navigační chyba'",
 "Tags: \n

In [ ]:
concatenated_text

'Document: gen_2_cz.pdf\n01 MAY 14 (1) Deset tipů VFR-GEN-2- 1\nDocument: gen_2_cz.pdf\nVFR příručka - Česká republikaGEN-2 DESET TIPŮ JAK SE VYHNOUT PROBLÉMŮM\nDocument: gen_2_cz.pdf\n1)Uvědomte si, že dokonalá znalost leteckých předpisů, pravidel a postupů, spolu se správným\nDocument: gen_2_cz.pdf\nplánováním a předletovou přípravou, je základním předpokladem k bezpečnému provedení\nDocument: gen_2_cz.pdf\nletu!\nDocument: gen_2_cz.pdf\nAIP ČR: http://lis.rlp.cz/ais_data/www_main_control/frm_cz_aip.htm\nDocument: gen_2_cz.pdf\nLetecké předpisy: http://lis.rlp.cz/predpisy/predpisy/index.htm\nDocument: gen_2_cz.pdf\n2)Jestliže plánujete trať, pamatujte, že meteorologické nebo provozní podmínky vám nemusí\nDocument: gen_2_cz.pdf\numožnit dodržet původně plánovanou trať a proto nezapomeňte při výpočtech zohlednit\nDocument: gen_2_cz.pdf\ncelkově delší čas letu, zajistit dostatečnou zásobu paliva na delší trať a připravit si postupy\nDocument: gen_2_cz.pdf\npro přistání na náhradních let

Text splitter by chunks

In [ ]:
texts = text_splitter.create_documents([concatenated_text])

In [ ]:
texts[21]

Document(page_content='taxi\nDocument: ad-lkpa_text_cz.pdf\n15 OCT 15 (1) Polička VFR-AD-LKPA-TEXT- 1\nDocument: ad-lkpa_text_cz.pdf\nVFR příručka - Česká republika1 PRAVIDLA A OMEZENÍ MÍSTNÍHO LETOVÉHO PROVOZU\nDocument: ad-lkpa_text_cz.pdf\n1.1 Odklízení sněhu není zajišťováno.\nDocument: ad-lkpa_text_cz.pdf\n1.2 RWY jsou po dlouhotrvajících deštích nezpůsobilé.\nDocument: ad-lkpa_text_cz.pdf\n1.3 Výška letištních okruhů je 2953 ft/900 m AMSL.\nDocument: ad-lkpa_text_cz.pdf\n1.4 Přílet letadla bez obousměrného radiového spojení je možný pouze po dohodě s\nDocument: ad-lkpa_text_cz.pdf\nprovozovatelem AD.\nDocument: ad-lkpa_text_cz.pdf\n1.5 Letištní provozní zóna (ATZ) je na jihu ohraničena prostorem LKP7.\nDocument: ad-lkpa_text_cz.pdf\n2 DOPLŇUJÍCÍ INFORMACE\nDocument: ad-lkpa_text_cz.pdf\n2.1 Možná činnost modelářů v blízkosti THR RWY 15.\nDocument: ad-lkpa_text_cz.pdf\n2.2 Přílet/odlet mimo provozní dobu letiště je povolen pouze po předchozí domluvě s\nDocument: ad-lkpa_text_cz.pd

Build a vectore store

In [ ]:
# Build vectorstore and keep the metadata
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# Instantiate OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# Create the Vector Store
vectorstore = Chroma.from_documents(documents=texts, embedding=embeddings)

In [ ]:
# Doing similarity search  using query
query = "frekvence"
matching_docs = vectorstore.similarity_search(query)

matching_docs[0]

Document(page_content='Typ Volací značka FREQ Prostor odpovědnosti/působnosti Poznámky\nFIC Praha INFORMATION 126,1001SECTOR ČECHY WEST H 24\nFIC Praha INFORMATION 136,1751SECTOR ČECHY EAST H 24\nFIC Praha INFORMATION 136,2751SECTOR MORAVA H 24\nVOLMET Praha VOLMET 128,605Mezinárodní vysílání:\nBERLIN/Schonefeld\nBRATISLAVA/Ivanka\nBUDAPEST/Ferihegy\nFRANKFURT/Main\nMUNCHEN\nPRAHA/Ruzyně\nWARSZAWA/Okecie\nZURICHH 24\nEN\nVOLMET Praha VOLMET 125,5251Vnitrostátní vysílání:\nBRNO/Tuřany\nKARLOVY VARY\nOSTRAVA/Mošnov\nPARDUBICE\nPRAHA/Ruzyně\nKUNOVICEH 24\nEN\nACC Praha RADAR 120,2751SECTOR WL H 24\nACC Praha RADAR 127,1251SECTOR SL H 24\nACC Praha RADAR 127,8251SECTOR NL H 24\n7.4 Skupinové kmitočty\nFREQ Účel ÚčelVýškové\nvymezeníHorizontální vymezení Poznámky\n130,930SEKTOR ČECHY WEST\nHLAVNÍ FREQ\n134,735SEKTOR ČECHY WEST\nZÁLOŽNÍ FREQ\n135,410SEKTOR ČECHY EAST +\nSEKTOR MORAVA\nHLAVNÍ FREQ\n136,085Kluzáky\n"Letadlo-letadlo"Do FL 95\nSEKTOR ČECHY EAST +\nSEKTOR MORAVA\nZÁLOŽNÍ FREQMimo

Using LLM model

In [ ]:
# load the LLM model
from langchain.chat_models import ChatOpenAI
model_name = "gpt-4"
llm = ChatOpenAI(model_name=model_name, temperature=0.7)


# Using q&a chain to get the answer for our query
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

# write your query and perform similarity search to generate an answer
query = "Jaky je poplatek na letisti LKLT"
matching_docs = vectorstore.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query, prompt="You re helpful co-pilot.")
answer



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Document: ad-lkmt_text_cz.pdf
lety 200 CZK / plné přistání, letmé přistání nebo přelet.
Document: ad-lkmt_text_cz.pdf
V době 2200-0400 (2100-0300) se sleva na výcvikové lety neposkytuje.
Document: ad-lkmt_text_cz.pdf
3.2 Parkovací poplatky
Document: ad-lkmt_text_cz.pdf
odbavovací plocha (za hodinu a tunu MTOW) 17,00
Document: ad-lkmt_text_cz.pdf
První dvě hodiny zdarma pro ACFT se sedadlovou kapacitou rovnou nebo větší než 200.
Document: ad-lkmt_text_cz.pdf
3.3 Poplatky za použití letiště cestujícími
Document: ad-lkmt_text_cz.pdf
Vnitrostátní lety (za cestujícího) 490,00
Document: ad-lkmt_text_cz.pdf
Mezinárodní lety (za cestujícího) 490,00
Document: ad-lkmt_text_cz.pdf
Transfer (za cestujícího) 300,

"I'm sorry, but the documents provided do not contain information on the fees at the LKLT airport."